## Bibliothek importieren und Datensatz laden

In [ ]:
import deepwaveform as dwf
import torch
df = dwf.load_dataset("../data/elbabschnitt.txt", wv_cols=list(range(64)))
df

## Datensatz in eine passende Form bringen und Trainingsprozess initialisieren

In [ ]:
train_df = df.sample(frac=0.8)              # Teilt den gesamten Datensatz in zwei Hälften
test_df = df.drop(train_df.index)           # 
train_ds = dwf.WaveFormDataset(train_df)    # Bringt den Datensatz in eine passende Form
test_ds = dwf.WaveFormDataset(test_df)      # 
model = dwf.ConvNet(output_dimension=2)     # Convolutional Neural Network initialisieren
trainer = dwf.Trainer(model,                # Das Modell, das trainiert werden soll
                      train_ds,             # Der Datensatz, der dafür verwendet werden soll
                      optimizer=None,       # Der Optimierer, der den Trainingsprozess regelt (für 'None' wird ein Standardoptimierer verwendet)
                      batch_size=1024,      # Wie viele Beispiele in einen Optimierungsschritt einfließen sollen
                      epochs=10)            # Wie oft über den kompletten Datensatz iteriert werden soll

## Trainieren und Zwischenergebnisse visualisieren 

In [ ]:
stats = []                                  # Initialisiert eine leere Liste, in der die Ergebnisse akkumuliert werden
for epoch, result in enumerate(trainer.train_classifier(), start=1):   # Trainiert das Modell
    print("epoch=%s E[loss]=%.3f Var[loss]=%.3f" % (str(epoch).zfill(3), result["meanloss"], result["varloss"]))
    stats.append(result)                    # Akkumuliert Zwischenergebnisse
dwf.plot_training_progress(stats)           # Plotted die Zwischenergebnisse

Der Trainingsfortschritt über die Epochen - je niedriger, desto weniger Fortschritt wird in jeder Iteration erzielt. Das ist ein Zeichen dafür, dass die Güte der Parameter zu einem lokalen Minimum konvergiert.

---

## Plotten der Confusion Matrix
Die "Confusion Matrix" zeigt an, wie oft das Modell welche Klassen miteinander verwechselt. Die Einträge können als Wahrscheinlichkeiten verstanden werden. Beispielsweise ist der obere rechte Eintrag die Wahrscheinlichkeit, dass das Modell eine Wasser-Waveform bekommt und diese dann als "Land" klassifiziert.

In [ ]:
dwf.plot_confusion_matrix(model, test_ds, class_label_mapping=["Land", "Water"])

## Speichern und Laden des Modells

In [ ]:
torch.save(model.state_dict(), "model_classifier.pt")               # Speichert die Parameter des Modells

In [ ]:
loaded_model = dwf.ConvNet(output_dimension=2)                      # Erstellt ein neues Modell mit zufälligen Parametern
dwf.plot_confusion_matrix(loaded_model, test_ds)                    # ...das nicht besonders gut zwischen Land und Wasser unterscheiden kann
loaded_model.load_state_dict(torch.load("model_classifier.pt"))     # lädt die gespeicherten Parameter in das neue Modell
loaded_model.eval()
dwf.plot_confusion_matrix(loaded_model, test_ds)                    # ...welches im Anschluss deutlich besser klassifizieren kann.

## Datensatz prozessieren
Wir können das trainierte Modell auf einen unbekannten Datensatz ansetzen und so Vorhersagen über die Objektklassen treffen. 

In [ ]:
unknown_df = dwf.load_dataset("../data/mit_uferuebergang.txt")

loaded_model.process_dataframe(unknown_df,                                      # Der Datensatz, der annotiert werden soll
                               wv_cols=list(range(64)),                         # Die Spaltennamen, die die Waveform angeben
                               class_label_mapping=["Land", "Water"],           # Zu den Klassen korrepondierende Labelnamen
                               predicted_column="Predicted")                    # Spaltenname für wahrscheinlichste Klasse
unknown_df

## Vom Modell klassifizierten Datensatz anzeigen lassen
Die Farben der verschiedenen Klassen können angegeben werden. Abhängig von der Sicherheit des Modells werden die Farben (linear) interpoliert.

In [ ]:
 dwf.plot_pcl_prediction(unknown_df,                                # Der geladene Datensatz
                        plotsize=12,                                # Größe des Plots
                        probabilities_col=["Land", "Water"],        # Spalten, die die Modellvorhersage zu den entsprechenden Klassen enthalten
                        colors=["green","blue"],                    # Farben, die den Klassen gegeben werden sollen
                        xcol="x",                                   # Spalten, die x,y und z-Positionen der Punkte enthalten
                        ycol="y",
                        zcol="z")